# S2S Skill

What was the forecast skill of S2S models? 
Did grid-cell or spatial bias correction schemes improve results?

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefigs = True

Load some plotting parameters from file

In [ ]:
%run ../config/PlotParameters.py
print(extent)
print(cmap)

In [ ]:
obs_url = 'http://iridl.ldeo.columbia.edu/home/.agmunoz/.Paraguay/.Obs/.Obsrain_90thpctl/dods'
obs_exceed = xr.open_dataarray(obs_url)
obs_exceed = obs_exceed.isel(T=(obs_exceed['T'].size - 1))
hatch0 = np.ma.masked_invalid(obs_exceed.values)
hatch = np.ma.masked_less(hatch0, 0.9)
Xh, Yh = np.meshgrid(obs_exceed.X, obs_exceed.Y)

In [ ]:
forecast = xr.open_dataset('../data/accessed/s2s_forecast.nc')
skill = xr.open_dataset('../data/accessed/s2s_skill.nc')
ignorance = xr.open_dataset('../data/accessed/s2s_ignorance.nc')

In [ ]:
model_names = forecast.model.values.tolist()

In [ ]:
def get_forecast(model, type):
    base_url = 'http://iridl.ldeo.columbia.edu/ds%3A/home/.agmunoz/.Paraguay/'
    url = base_url + '{}/'.format(model)
    if type == 'forecast':
        url += '.Odds_1-7Dec2015/dods'
    elif type == 'skill':
        url += '.2AFCProbFcsts/dods'
    elif type == 'ignorance':
        url += '.IGNProbFcsts/dods'
    else:
        raise ValueError('type is not valid')

    ds = xr.open_dataarray(url)
    ds = ds.sortby(['X', 'Y']) # be more consistent
    if ds.ndim > 2:
        ds = ds.isel(time=0).drop('time')

    ds.coords['model'] = model

    return(ds)

In [ ]:
ncols = len(model_names)
fig, axes = plt.subplots(
    ncols=len(model_names), nrows=3,
    subplot_kw={'projection': ccrs.PlateCarree()},
    figsize=(14,10), sharex=True, sharey=True
)
for i,m in enumerate(model_names):
    # Forecast for December 1-7 2015
    ax = axes[0, i]
    ax.set_title(m)
    ds = get_forecast(model=m, type='forecast')
    X,Y = np.meshgrid(ds.X, ds.Y)
    sub = ds.values
    sub = np.ma.masked_invalid(sub)
    sub = np.ma.masked_less(sub, 0)
    C1 = ax.pcolormesh(X, Y, sub, cmap="gist_rainbow", transform=ccrs.PlateCarree(), vmin=0, vmax=8)
    ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')
    # Ignorance Score
    ax = axes[1, i]
    ds = get_forecast(model=m, type='ignorance')
    X,Y = np.meshgrid(ds.X, ds.Y)
    sub = ds.values
    sub = np.ma.masked_invalid(sub)
    sub = np.ma.masked_less(sub, 0)
    C2 = ax.pcolormesh(X, Y, sub, transform=ccrs.PlateCarree(), cmap="gist_rainbow", vmin=0, vmax=2.5)
    ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')
    # 2AFC Skill Score
    ax = axes[2, i]
    ds = get_forecast(model=m, type='skill')
    X,Y = np.meshgrid(ds.X, ds.Y)
    sub = ds.values
    sub = np.ma.masked_invalid(sub)
    sub = np.ma.masked_less(sub, 0)
    C3 = ax.pcolormesh(X, Y, sub, transform=ccrs.PlateCarree(), cmap="PuOr", vmin=0, vmax=100)
    ax.pcolor(Xh, Yh, hatch, facecolor='none', edgecolors='k')

viz.FormatAxes(axes, ticks=[np.linspace(-180, 180, 73), np.linspace(-90, 90, 37)], 
               extent = [np.min(X), np.max(X), np.min(Y), np.max(Y)])
fig.tight_layout()
fig.subplots_adjust(right=0.94)
cax1 = fig.add_axes([0.97, 0.7, 0.015, 0.25])
cax2 = fig.add_axes([0.97, 0.38, 0.015, 0.25])
cax3 = fig.add_axes([0.97, 0.05, 0.015, 0.25])
# Color bar for Odds Ratio
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.update_ticks()
cbar1.set_label(r'Odds Ratio', rotation=270)
cbar1.ax.get_yaxis().labelpad = 15
# Color bar for 2AFC
cbar2 = fig.colorbar(C2, cax=cax2)
cbar2.set_label(r'Ignorance Score', rotation=270)
cbar2.ax.get_yaxis().labelpad = 15
# Color bar for Ignorance
cbar3 = fig.colorbar(C3, cax=cax3)
cbar3.set_label(r'2AFC Skill Score', rotation=270)
cbar3.ax.get_yaxis().labelpad = 15
if savefigs:
    fig.savefig("../figs/ForecastSkill.pdf", bbox_inches="tight")